## 过拟合与欠拟合现象
以线性回归模型为基础，拟合目标是$$y = 5 + 1.2x - 3.4\frac{x^2}{2!} + 5.6 \frac{x^3}{3!} + \epsilon \text{ where }
\epsilon \sim \mathcal{N}(0, 0.1^2).$$，实验通过变化不同的迭代次数以及最高幂次来调整模型复杂度，数据集可以自行生成，注意区分训练集和验证集

In [ ]:
# 生成模拟数据，训练集100条，验证集100条
